In [1]:
from enum import Enum
import random
from tqdm import tqdm

class NodeType(Enum):
    USER = 0
    MOVIE = 1

node_map = []
graph = []
test_graph = []
test_size = 0.1

user_map = {}
movie_map = {}

last_movie_idx = 0
with open("combined_data_1.txt") as data_file:
    for line in tqdm(data_file):
        line = line.strip()
        if line.endswith(":"):
            movie_id = int(line[:-1])
            
            node_map.append((NodeType.MOVIE, movie_id))
            graph.append([])
            test_graph.append([])
            
            movie_map[movie_id] = len(node_map) - 1
            last_movie_idx = len(node_map) - 1
        else:
            node_info = line.split(",")
            user_id = int(node_info[0])
            rating = int(node_info[1])
            
            if rating > 3:
                if user_id not in user_map.keys():
                    node_map.append((NodeType.USER, user_id))
                    graph.append([])
                    test_graph.append([])
                    
                    user_map[user_id] = len(node_map) - 1
                    
                user_node_idx = user_map[user_id]
                
                if random.uniform(0, 1) > test_size:
                    graph[last_movie_idx].append(user_node_idx)
                    graph[user_node_idx].append(last_movie_idx)
                else:
                    test_graph[last_movie_idx].append(user_node_idx)
                    test_graph[user_node_idx].append(last_movie_idx)

24058263it [00:37, 644300.09it/s]


In [2]:
len(user_map)

458436

In [3]:
epochs = 5
t = 30
w = 10
latent_dim = 128

corpus = []

nodes = list(range(len(node_map)))
for _ in tqdm(range(epochs), total = epochs):
    random.shuffle(nodes)
    for node in nodes:
        walk = [node]
        for i in range(1, t):
            if len(graph[walk[-1]]) == 0:
                break
            walk.append(random.choice(graph[walk[-1]]))
        corpus.append([str(word) for word in walk])

100%|██████████| 5/5 [01:18<00:00, 15.74s/it]


In [4]:
from gensim.models import Word2Vec

model = Word2Vec(corpus, size = latent_dim, window = w, min_count = 0, sg = 1, hs = 1, workers = 4)

In [41]:
rand_user = 0
while node_map[rand_user][0] != NodeType.USER or len(test_graph[rand_user]) == 0:
    rand_user = random.randrange(len(node_map))

similar = model.wv.most_similar(positive = [str(rand_user)], topn = 200)
similar_movies = [int(x[0]) for x in similar if node_map[int(x[0])][0] == NodeType.MOVIE]
recommendations = [x for x in similar_movies if x not in graph[rand_user]]
print(recommendations)
print(similar_movies)
print(test_graph[rand_user])
print(graph[rand_user])
print(rand_user)
n_guesses = len([x for x in recommendations if x in test_graph[rand_user]])
print(n_guesses / len(test_graph[rand_user]))  # Procenat pozeljnih filmova koji su predlozeni
#print(n_guesses / len(recommendations))  # Procenat predlozenih filmova koji su pozeljni

[302720, 382080]
[446753, 415003, 429777, 320625, 302720, 382080]
[434687, 439763]
[307777, 320625, 333347, 333995, 369438, 406788, 415003, 416463, 427355, 428752, 429777, 436373, 446753]
307968
0.0
